<h1 style="color:blue;text-align:center;">Lab 1 - Scraping and NLP Pipeline</h1>

<hr style="border:2px solid blue;">

<h3 style="color:blue;">Realised by: <strong style="color:black;">Ouahid Mariyam</strong></h3>
 

<h3 style="color:blue;">Guided by: <strong style="color:black;">Pr . ELAACHAk LOTFI</strong></h3> 
   

<p><strong>Objective :</strong> The main purpose behind this lab is to get familiar with Scraping and NLP
 Pipeline.</p>
 
 
 
 <p>This lab effectively combines data scraping with NLP techniques to collect, preprocess, and analyze text data from the web. The scraping process enables the extraction of relevant information from web pages, while the NLP pipeline facilitates text processing and analysis for various applications such as text classification, sentiment analysis, and information extraction. Overall, this lab provides practical experience in utilizing web scraping and NLP techniques for data collection and analysis tasks. </p>


#       Summary:

1. [Data Loading](#data-loading)
2. [NLP Pipeline](#nlp-pipeline)
    1. [Text Cleaning](#text-cleaning)
    2. [Tokenization](#tokenization)
    3. [Stop words](#stop-words)
    4. [Discretization](#discretization)
    5. [Normalization](#nor)
    6. [ Stemming and Lemmatization](#SL)
3. [Parts of Speech (POS)](#part)
4. [Named Entity Recognition(NER)](#ner)
5. [Lab Reflection](#lab)

## 1. Data Loading <a id='data-loading'></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd

In [2]:
def scrape_recipe_data(url):
    titles = []
    properties = []
    chefs = []
    links = []
    ingredients = []
    recipes = []

    # Retrieve the content of the web page
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, "lxml")

    # Find the elements on the web page containing the data
    titles_tags = soup.find_all("h2", {"class": "mb-0 semibold-font"})
    properties_tags = soup.find_all("div", {"class": "tabkha-details rounded-pill py-1 me-1 medium-font"})
    chef_tags = soup.find_all("h3", {"class": "pt-5 recipe-chef-name medium-font"})
    div_tags = soup.find_all("a", {"class": "recipe-real-image"})

    # Loop to extract and store the data
    for i in range(len(titles_tags)):
        titles.append(titles_tags[i].text.strip())
        properties.append(properties_tags[i].text.strip())
        # Check if chef tag exists before accessing its text
        if i < len(chef_tags):
            chefs.append(chef_tags[i].text.strip())
        else:
            chefs.append("Unknown Chef")
        links.append(div_tags[i].attrs['href'])

    # Loop to extract ingredients and recipes from each recipe's page
    for link in links:
        result = requests.get(link)
        src = result.content
        soup = BeautifulSoup(src, "lxml")
        ingredients_tags = soup.find("div", {"class": "row px-0 px-lg-3 ingredients"})
        if ingredients_tags:
            ingredients.append(ingredients_tags.text.strip())
        else:
            ingredients.append("Ingredients not available")
        recipes_tags = soup.find("div", {"class": "row px-0 px-lg-3 preparationWay"})
        if recipes_tags:
            recipes.append(recipes_tags.text.strip())
        else:
            recipes.append("Recipes not available")

    return titles, properties, chefs, ingredients, recipes


In [3]:
def save_to_mongodb(data):
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client["myScraping"]
    collection = db["mydb"]
    for i in range(len(data[0])):
        recipe = {
            "title": data[0][i],
            "properties": data[1][i],
            "chef": data[2][i],
            "ingredients": data[3][i],
            "recipes": data[4][i]
        }
        collection.insert_one(recipe)
    client.close()


In [4]:
# Scrape and save cake recipes
cake_data = scrape_recipe_data("https://www.fatafeat.com/recipes-list/207106-%D9%83%D9%8A%D9%83")
save_to_mongodb(cake_data)

# Scrape and save pizza recipes
pizza_data = scrape_recipe_data("https://www.fatafeat.com/search?term=%D8%A8%D9%8A%D8%AA%D8%B2%D8%A7")
save_to_mongodb(pizza_data)

# Scrape and save cookies recipes
cookies_data = scrape_recipe_data("https://www.fatafeat.com/recipes-list/207104-%D9%83%D9%88%D9%83%D9%8A%D8%B2")
save_to_mongodb(cookies_data)

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["myScraping"]
collection = db["mydb"]

In [5]:
# Fetch data from MongoDB
cursor = collection.find()

In [6]:
# Convert the cursor to a list of dictionaries
data = list(cursor)

In [7]:
# Create a DataFrame from the data
df = pd.DataFrame(data)

In [8]:
# Close the MongoDB connection
client.close()

In [9]:
# Display the DataFrame
df

,_id,title,properties,chef,ingredients,recipes
0,66135d7bccad4da1905a3eb0,تشيز كيك قليل الدسم,سهلة,ليلى فتح الله,180 \n\n\nجرام بسكويت عادي\n\n\n\n\n\n\n 90 \n...,1\n\nيحمى الفرن إلى 160 درجة مئوية (بدون مروحة...
1,66135d7bccad4da1905a3eb1,كيك الموز بالتوفي,سهلة,منال العالم,1 1/2 \n\n\nكوب دقيق\n\n\n\n\n\n\n 1 \n\n\nملع...,1\n\nيثبت الرف الشبكي في وسط الفرن. يسخن الفرن...
2,66135d7bccad4da1905a3eb2,كيك ماربل بالكاكاو,سهلة,منال العالم,3 \n\n\nكوب دقيق\n\n\n\n\n\n\n 3 \n\n\nملعقة ص...,1\n\nيحضر قالب ذو فتحة من الوسط سعة 12 كوب. يد...
3,66135d7bccad4da1905a3eb3,كيك القرع واللوز,سهلة,وصفات فتافيت,125 \n\n\nجرام دقيق اللوز\n\n\n\n\n\n\n 25 \n\...,1\n\nيخلط دقيق اللوز والطحين متعدد الاستعمالات...
4,66135d7bccad4da1905a3eb4,كعكة‭ ‬الجواهر,سهلة,وصفات فتافيت,0 \n\n\nعبوة تحتوي على 4 أنواع مختلفة من الجيل...,Recipes not available
5,66135d7bccad4da1905a3eb5,كعكة‭ ‬البلاك‭ ‬فورست,سهلة,وصفات فتافيت,200 \n\n\nجم الدقيق\n\n\n\n\n\n\n 50 \n\n\nجم ...,1\n\nتُخفق‭ ‬الزبدة‭ ‬مع‭ ‬السكر‭ ‬حتى‭ ‬يتكون...
6,66135d7bccad4da1905a3eb6,مخبوز البانوفى بالموز ( كيك...,متوسطة,أفنان,190 \n\n\nجرام دقيق\n\n\n\n\n\n\n 0 \n\n\nملعق...,1\n\nلعمل الصوص : يرفع السكر مع الماء على النا...
7,66135d7bccad4da1905a3eb7,كيك‭ ‬التمر,سهلة,وصفات فتافيت,2 \n\n\n بيضة\n\n\n\n\n\n\n 3 \n\n\n أكواب من ...,1\n\nتُنخل‭ ‬المكونات‭ ‬الجافة‭ ‬معاً‭ ‬فيما‭ ...
8,66135d7bccad4da1905a3eb8,كيكة السميد,سهلة,منال العالم,1/2 \n\n\nكوب دقيق\n\n\n\n\n\n\n 2 \n\n\nملعقة...,1\n\nيسخن الفرن لدرجة حرارة 180. يحضر قالب مست...
9,66135d7bccad4da1905a3eb9,كاب كيك بشوكولاتة فيريرو,متوسطة,وصفات فتافيت,125 \n\n\nجرام زبدة\n\n\n\n\n\n\n 1 1/2 \n\n\n...,1\n\nيخفق البيض والسكر والفانيليا حتى يصبح الم...


## 2. NLP Pipeline <a id='nlp-pipeline'></a>

### 2.1 Text Cleaning <a id='text-cleaning'></a>

In [10]:
import re

# Arabic Text Cleaning
def clean_arabic_text(text):
    # Remove non-Arabic characters and extra spaces
    cleaned_text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Apply cleaning to each column that contains text
df['title'] = df['title'].apply(clean_arabic_text)
df['properties'] = df['properties'].apply(clean_arabic_text)
df['ingredients'] = df['ingredients'].apply(clean_arabic_text)
df['recipes'] = df['recipes'].apply(clean_arabic_text)

In [11]:
# Display the DataFrame
df

,_id,title,properties,chef,ingredients,recipes
0,66135d7bccad4da1905a3eb0,تشيز كيك قليل الدسم,سهلة,ليلى فتح الله,جرام بسكويت عادي جرام زبدة جرام جبنة الكريم جر...,يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...
1,66135d7bccad4da1905a3eb1,كيك الموز بالتوفي,سهلة,منال العالم,كوب دقيق ملعقة كبيرة بيكنج باودر ملعقة صغيرة م...,يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...
2,66135d7bccad4da1905a3eb2,كيك ماربل بالكاكاو,سهلة,منال العالم,كوب دقيق ملعقة صغيرة بيكنج باودر ملعقة صغيرة م...,يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...
3,66135d7bccad4da1905a3eb3,كيك القرع واللوز,سهلة,وصفات فتافيت,جرام دقيق اللوز جرام طحين متعدد الاستعمالات مل...,يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...
4,66135d7bccad4da1905a3eb4,كعكة الجواهر,سهلة,وصفات فتافيت,عبوة تحتوي على أنواع مختلفة من الجيلاتين المنك...,
5,66135d7bccad4da1905a3eb5,كعكة البلاك فورست,سهلة,وصفات فتافيت,جم الدقيق جم الكاكاو البودرة ملعقة صغيرة البيك...,تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...
6,66135d7bccad4da1905a3eb6,مخبوز البانوفى بالموز كيك,متوسطة,أفنان,جرام دقيق ملعقة صغيرة و نصف بيكنج باودر جرام ز...,لعمل الصوص يرفع السكر مع الماء على النار درجة ...
7,66135d7bccad4da1905a3eb7,كيك التمر,سهلة,وصفات فتافيت,بيضة أكواب من الدقيق ملعقة صغيرة برش الليمون م...,تنخل المكونات الجافة معا فيما عدا السكر توضع ب...
8,66135d7bccad4da1905a3eb8,كيكة السميد,سهلة,منال العالم,كوب دقيق ملعقة صغيرة بيكنج باودر كوب سميد ناعم...,يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...
9,66135d7bccad4da1905a3eb9,كاب كيك بشوكولاتة فيريرو,متوسطة,وصفات فتافيت,جرام زبدة كوب سكر ملعقة صغيرة فانيليا حبة بيض ...,يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...


### 2.2 Tokenization <a id='tokenization'></a>

In [12]:
from nltk.tokenize import WordPunctTokenizer

# Create a tokenizer instance
tokenizer = WordPunctTokenizer()

# Function to tokenize a single piece of text
def tokenize_text(text):
    return tokenizer.tokenize(text)

df['title_tokens_treebank'] = df['title'].apply(tokenizer.tokenize)
df['properties_tokens_treebank'] = df['properties'].apply(tokenizer.tokenize)
df['ingredients_tokens_treebank'] = df['ingredients'].apply(tokenizer.tokenize)
df['recipes_tokens_treebank'] = df['recipes'].apply(tokenizer.tokenize)

df.head()

,_id,title,properties,chef,ingredients,recipes,title_tokens_treebank,properties_tokens_treebank,ingredients_tokens_treebank,recipes_tokens_treebank
0,66135d7bccad4da1905a3eb0,تشيز كيك قليل الدسم,سهلة,ليلى فتح الله,جرام بسكويت عادي جرام زبدة جرام جبنة الكريم جر...,يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...,"[تشيز, كيك, قليل, الدسم]",[سهلة],"[جرام, بسكويت, عادي, جرام, زبدة, جرام, جبنة, ا...","[يحمى, الفرن, إلى, درجة, مئوية, بدون, مروحة, ت..."
1,66135d7bccad4da1905a3eb1,كيك الموز بالتوفي,سهلة,منال العالم,كوب دقيق ملعقة كبيرة بيكنج باودر ملعقة صغيرة م...,يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...,"[كيك, الموز, بالتوفي]",[سهلة],"[كوب, دقيق, ملعقة, كبيرة, بيكنج, باودر, ملعقة,...","[يثبت, الرف, الشبكي, في, وسط, الفرن, يسخن, الف..."
2,66135d7bccad4da1905a3eb2,كيك ماربل بالكاكاو,سهلة,منال العالم,كوب دقيق ملعقة صغيرة بيكنج باودر ملعقة صغيرة م...,يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...,"[كيك, ماربل, بالكاكاو]",[سهلة],"[كوب, دقيق, ملعقة, صغيرة, بيكنج, باودر, ملعقة,...","[يحضر, قالب, ذو, فتحة, من, الوسط, سعة, كوب, يد..."
3,66135d7bccad4da1905a3eb3,كيك القرع واللوز,سهلة,وصفات فتافيت,جرام دقيق اللوز جرام طحين متعدد الاستعمالات مل...,يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...,"[كيك, القرع, واللوز]",[سهلة],"[جرام, دقيق, اللوز, جرام, طحين, متعدد, الاستعم...","[يخلط, دقيق, اللوز, والطحين, متعدد, الاستعمالا..."
4,66135d7bccad4da1905a3eb4,كعكة الجواهر,سهلة,وصفات فتافيت,عبوة تحتوي على أنواع مختلفة من الجيلاتين المنك...,,"[كعكة, الجواهر]",[سهلة],"[عبوة, تحتوي, على, أنواع, مختلفة, من, الجيلاتي...",[]


### 2.3 Stop words <a id='stop-words'></a>

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download the stopwords list for Arabic
nltk.download('stopwords')

# Load Arabic stopwords
arabic_stop_words = set(stopwords.words('arabic'))

# Function to tokenize a single piece of text and remove Arabic stopwords
def tokenize_and_remove_stopwords(text):
    # Tokenize the text
    word_tokens = word_tokenize(text)
    # Remove Arabic stopwords
    filtered_sentence = [w for w in word_tokens if not w in arabic_stop_words]
    return filtered_sentence

# Function to remove stopwords from a piece of text
def remove_stopwords(text, stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)

# Assuming df is your DataFrame with columns 'title', 'properties', 'ingredients', and 'recipes'

# Apply remove_stopwords function to each column
df['title_tokens'] = df['title'].apply(lambda x: remove_stopwords(x, arabic_stop_words))
df['properties_tokens'] = df['properties'].apply(lambda x: remove_stopwords(x, arabic_stop_words))
df['ingredients_tokens'] = df['ingredients'].apply(lambda x: remove_stopwords(x, arabic_stop_words))
df['recipes_tokens'] = df['recipes'].apply(lambda x: remove_stopwords(x, arabic_stop_words))

print(df[['recipes', 'recipes_tokens']].head(20))


                                              recipes  \
0   يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1   يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2   يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3   يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                       
5   تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6   لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7   تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8   يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9   يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   
10  يخلط السميد والطحين والكركم والزعفران والباكين...   
11  تخفق الزبدة مع السكر والفانيليا حتى يصبح المزي...   
12  يسخن الفرن لدرجة حرارة يستعمل قالب ذو فتحة في ...   
13  يسخن الفرن على درجة حرارة درجة مئوية في وعاء ك...   
14  يضرب في وعاء معجون الفستق الحلبي والجلوكوز لمد...   
15  في وعاء عميق نقوم بوضع اللبنة والحليب ونخلطهم ...   
16  يسخن الفرن إلى درجة مئوية غ

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2.4 Discretization <a id='discretization'></a>

In [15]:
# Define the interval borders for the length of the texts
bins = [0, 180, 280, 380]  # Adjust these values according to your data
labels = ['short', 'medium', 'long']

# Discretize the length of 'ingredients'
df['ingredients_length'] = df['ingredients'].apply(len)

df['ingredients_length_category'] = pd.cut(df['ingredients_length'], bins=bins, labels=labels)

# Display the DataFrame to verify
print(df[['ingredients', 'ingredients_length', 'ingredients_length_category']].head(10))


                                         ingredients  ingredients_length  \
0  جرام بسكويت عادي جرام زبدة جرام جبنة الكريم جر...                 111   
1  كوب دقيق ملعقة كبيرة بيكنج باودر ملعقة صغيرة م...                 244   
2  كوب دقيق ملعقة صغيرة بيكنج باودر ملعقة صغيرة م...                 255   
3  جرام دقيق اللوز جرام طحين متعدد الاستعمالات مل...                 347   
4  عبوة تحتوي على أنواع مختلفة من الجيلاتين المنك...                 362   
5  جم الدقيق جم الكاكاو البودرة ملعقة صغيرة البيك...                 198   
6  جرام دقيق ملعقة صغيرة و نصف بيكنج باودر جرام ز...                 287   
7  بيضة أكواب من الدقيق ملعقة صغيرة برش الليمون م...                 268   
8  كوب دقيق ملعقة صغيرة بيكنج باودر كوب سميد ناعم...                 226   
9  جرام زبدة كوب سكر ملعقة صغيرة فانيليا حبة بيض ...                 268   

  ingredients_length_category  
0                       short  
1                      medium  
2                      medium  
3                        long  
4  

### 2.5 Normalization <a id='nor'></a>

In [16]:
# Normalisation du texte
def normalize_arabic_text(text):
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    text = re.sub(r'أ|إ|آ', 'ا', text)
    return text


df['recipes_norm'] = df['recipes'].apply(normalize_arabic_text)
print(df[['recipes', 'recipes_norm']].head(20))

                                              recipes  \
0   يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1   يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2   يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3   يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                       
5   تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6   لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7   تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8   يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9   يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   
10  يخلط السميد والطحين والكركم والزعفران والباكين...   
11  تخفق الزبدة مع السكر والفانيليا حتى يصبح المزي...   
12  يسخن الفرن لدرجة حرارة يستعمل قالب ذو فتحة في ...   
13  يسخن الفرن على درجة حرارة درجة مئوية في وعاء ك...   
14  يضرب في وعاء معجون الفستق الحلبي والجلوكوز لمد...   
15  في وعاء عميق نقوم بوضع اللبنة والحليب ونخلطهم ...   
16  يسخن الفرن إلى درجة مئوية غ

### 2.6  Stemming and Lemmatization <a id='SL'></a>

In [17]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk


# Download the WordNet corpus for stemming
nltk.download('wordnet')

# Load Arabic stopwords
arabic_stop_words = set(stopwords.words('arabic'))

# Create an object of class PorterStemmer
porter = PorterStemmer()

# Function to tokenize a single piece of text, remove Arabic stopwords, and perform stemming
def tokenize_remove_stopwords_and_stem(text):
    # Tokenize the text
    word_tokens = word_tokenize(text)
    # Remove Arabic stopwords
    filtered_sentence = [w for w in word_tokens if not w in arabic_stop_words]
    # Perform stemming
    stemmed_words = [porter.stem(w) for w in filtered_sentence]
    return stemmed_words

# Tokenize, remove Arabic stopwords, and perform stemming for each column
df['title_tokens_stemmed'] = df['title'].apply(tokenize_remove_stopwords_and_stem)
df['properties_tokens_stemmed'] = df['properties'].apply(tokenize_remove_stopwords_and_stem)
df['ingredients_tokens_stemmed'] = df['ingredients'].apply(tokenize_remove_stopwords_and_stem)
df['recipes_tokens_stemmed'] = df['recipes'].apply(tokenize_remove_stopwords_and_stem)

# Concatenate stemmed tokens from all columns
all_stemmed_tokens = df['title_tokens_stemmed'] + df['properties_tokens_stemmed'] + df['ingredients_tokens_stemmed'] + df['recipes_tokens_stemmed']

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
from nltk.stem import WordNetLemmatizer
import nltk

# Download the WordNet corpus for lemmatization
nltk.download('wordnet')

# Create an object of class WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to tokenize a single piece of text, remove Arabic stopwords, and perform lemmatization
def tokenize_remove_stopwords_and_lemmatize(text):
    # Tokenize the text
    word_tokens = word_tokenize(text)
    # Remove Arabic stopwords
    filtered_sentence = [w for w in word_tokens if not w in arabic_stop_words]
    # Perform lemmatization
    lemmatized_words = [lemmatizer.lemmatize(w) for w in filtered_sentence]
    return lemmatized_words

# Tokenize, remove Arabic stopwords, and perform lemmatization for each column
df['title_tokens_lemmatized'] = df['title'].apply(tokenize_remove_stopwords_and_lemmatize)
df['properties_tokens_lemmatized'] = df['properties'].apply(tokenize_remove_stopwords_and_lemmatize)
df['ingredients_tokens_lemmatized'] = df['ingredients'].apply(tokenize_remove_stopwords_and_lemmatize)
df['recipes_tokens_lemmatized'] = df['recipes'].apply(tokenize_remove_stopwords_and_lemmatize)

# Concatenate lemmatized tokens from all columns
all_lemmatized_tokens = df['title_tokens_lemmatized'] + df['properties_tokens_lemmatized'] + df['ingredients_tokens_lemmatized'] + df['recipes_tokens_lemmatized']

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
# Print stemmed tokens
print("Stemmed Tokens:")
print(all_stemmed_tokens)

# Print lemmatized tokens
print("\nLemmatized Tokens:")
print(all_lemmatized_tokens)


Stemmed Tokens:
0     [تشيز, كيك, قليل, الدسم, سهلة, جرام, بسكويت, ع...
1     [كيك, الموز, بالتوفي, سهلة, كوب, دقيق, ملعقة, ...
2     [كيك, ماربل, بالكاكاو, سهلة, كوب, دقيق, ملعقة,...
3     [كيك, القرع, واللوز, سهلة, جرام, دقيق, اللوز, ...
4     [كعكة, الجواهر, سهلة, عبوة, تحتوي, أنواع, مختل...
5     [كعكة, البلاك, فورست, سهلة, جم, الدقيق, جم, ال...
6     [مخبوز, البانوفى, بالموز, كيك, متوسطة, جرام, د...
7     [كيك, التمر, سهلة, بيضة, أكواب, الدقيق, ملعقة,...
8     [كيكة, السميد, سهلة, كوب, دقيق, ملعقة, صغيرة, ...
9     [كاب, كيك, بشوكولاتة, فيريرو, متوسطة, جرام, زب...
10    [سفوف, بالزعفران, سهلة, جرام, سميد, جرام, طحين...
11    [كيك, طبقات, آيس, كريم, متوسطة, جرام, زبدة, كو...
12    [كيك, مابدك, سهلة, كوب, دقيق, ملعقة, كبيرة, بي...
13    [كيندا, كيك, متقدمة, بياض, بيض, كوب, سكر, ابيض...
14    [كب, كيك, بالفستق, الحلبي, سهلة, جرام, معجون, ...
15    [بيتزا, التوست, سهلة, قطع, توست, كوب, لبنة, ما...
16    [بيتزا, الدجاج, متوسطة, عجينة, بيتزا, جاهزة, ك...
17    [بيتزا, الأم, العربية, متو

## 3. Parts of Speech (POS) <a id='part'></a>

In [19]:
# Assuming df is your DataFrame and 'recipes' is the column with text data
import stanza

# Load the POS tagging model for Arabic
nlp = stanza.Pipeline(lang='ar', processors='tokenize,pos')

# This function uses the loaded model to tag tokens
def pos_tag_arabic(tokens):
    doc = nlp(' '.join(tokens))
    return [(word.text, word.pos) for sent in doc.sentences for word in sent.words]

# Tokenize the 'recipes' column
df['recipes_tokenized'] = df['recipes'].apply(lambda x: x.split())

# Apply the POS tagging function to the tokenized text
df['recipes_pos_tags'] = df['recipes_tokenized'].apply(pos_tag_arabic)

# Check the results
print(df[['recipes', 'recipes_pos_tags']].head(10))


2024-04-08 03:01:47 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-08 03:01:48 INFO: Downloaded file to C:\Users\Administrator\stanza_resources\resources.json
2024-04-08 03:01:48 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 03:01:48 INFO: Loading these models for language: ar (Arabic):
| Processor | Package     |
---------------------------
| tokenize  | padt        |
| mwt       | padt        |
| pos       | padt_charlm |

2024-04-08 03:01:48 INFO: Using device: cpu
2024-04-08 03:01:48 INFO: Loading: tokenize
2024-04-08 03:01:50 INFO: Loading: mwt
2024-04-08 03:01:50 INFO: Loading: pos
2024-04-08 03:01:51 INFO: Done loading processors!


                                             recipes  \
0  يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                      
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6  لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7  تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8  يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9  يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   

                                    recipes_pos_tags  
0  [(يحمى, VERB), (الفرن, NOUN), (إلى, ADP), (درج...  
1  [(يثبت, VERB), (الرف, NOUN), (الشبكي, ADJ), (ف...  
2  [(يحضر, VERB), (قالب, NOUN), (ذو, DET), (فتحة,...  
3  [(يخلط, VERB), (دقيق, NOUN), (اللوز, NOUN), (و...  
4                                                 []  
5  [(تخفق, VERB), (الزبدة, NOUN), (مع, ADP), (الس... 

## 4. Named Entity Recognition(NER) <a id='ner'></a>

In [24]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Define function to apply NER on text
def apply_ner(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Apply NER to the 'recipes' column
df['recipes_ner'] = df['recipes'].apply(apply_ner)

# Print the results
print(df[['recipes', 'recipes_ner']].head(10))


                                             recipes  \
0  يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                      
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6  لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7  تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8  يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9  يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   

                                         recipes_ner  
0  [(كيك قياس, ORG), (الزبدة, ORG), (الذائبة ويضر...  
1  [(يثبت الرف, PERSON), (الزبدة, NORP), (ورق, OR...  
2  [(يحضر قالب ذو فتحة, ORG), (السمن, GPE), (ورق,...  
3  [(الزبدة والسكر, PERSON), (يضاف, GPE), (المقاد...  
4                                                 []  
5  [(تخفق الزبدة, PERSON), (السكر, GPE), (يضاف, G... 

## 5. Lab Reflection <a id='lab'></a>

In this lab, I learned about web scraping, which is the process of extracting data from websites. We used BeautifulSoup, a Python library, to scrape recipe data from the Fatafeat website. This involved parsing the HTML content of web pages to extract recipe titles, properties, chefs, links, ingredients, and recipes. We wrote a function called `scrape_recipe_data` to automate this process for cakes, pizzas, and cookies. The scraped data was then stored in MongoDB for easy access. Alongside web scraping, I also explored various natural language processing (NLP) techniques like text cleaning, tokenization, and named entity recognition (NER). This hands-on experience has enriched my understanding of data collection and analysis methods.
